In [1]:
import numpy as np
import simpleaudio as sa

from pythonosc.dispatcher import Dispatcher
from pythonosc.osc_server import BlockingOSCUDPServer

# Define tone playing function

In [2]:
def playTone(pitch=440, dur=0.2):
    # Sampling frequency 44.1kHz
    fs = 44100
    # Default note duration 0.2 sec
    tt = np.linspace(0, dur, int(dur*fs), False)

    # Generate a 440 Hz sine wave
    note = np.sin(pitch * tt * 2 * np.pi)

    # Ensure that highest value is in 16-bit range
    audio = note * (2**15 - 1) / np.max(np.abs(note))
    # Convert to 16-bit data
    audio = audio.astype(np.int16)

    # Start playback
    play_obj = sa.play_buffer(audio, 1, 2, fs)

    # Wait for playback to finish before exiting
    play_obj.wait_done()

In [3]:
def print_handler(address, *args):
    print(f"{address}: {args}")


def default_handler(address, *args):
    if len(args)>1:
        playTone(args[0], args[1])
    else:
        playTone(args)


dispatcher = Dispatcher()
dispatcher.map("/something/*", print_handler)
dispatcher.set_default_handler(default_handler)

ip = "127.0.0.1"
port = 1337

server = BlockingOSCUDPServer((ip, port), dispatcher)
server.serve_forever()  # Blocks forever

KeyboardInterrupt: 